In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pickle
import warnings
warnings.filterwarnings('ignore')
import re
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
ts_df = pd.read_csv('./data/SWOW_prediction/eval/time_series/ts_df.csv')

ts_df = ts_df.groupby(['words','year','property'])[['outputs_z','count']].mean().reset_index()
ts_df = ts_df.loc[ts_df['count'] >= 50]

In [4]:
category_norms = pd.read_csv('./data/Referential version_Item level data.csv')
category_norms = category_norms.loc[category_norms['prod.freq'] > 1]
category_norms.groupby('category')['category.member'].count().to_dict()


{'academic subject': 41,
 'alcoholic drink': 30,
 'animal': 69,
 'art form': 16,
 'artistic movement': 9,
 'bathroom fixture': 19,
 'bird': 38,
 'bird of prey': 8,
 'boat': 16,
 'body of water': 15,
 'book genre': 22,
 'breed of dog': 36,
 'building': 34,
 'building material': 18,
 'camping equipment': 25,
 "carpenter's tool": 16,
 'chemical element': 35,
 'citrus fruit': 11,
 'clothing': 47,
 'colour': 24,
 'cosmetic': 23,
 'crime': 25,
 'dairy product': 11,
 'day of the week': 7,
 'disease': 24,
 'drug': 24,
 'emotion': 27,
 'fabric': 15,
 'family relationship': 42,
 'farm animal': 18,
 'fish': 26,
 'flower': 18,
 'four-legged animal': 38,
 'four-wheeled vehicle': 13,
 'fraction': 24,
 'fruit': 40,
 'fuel': 14,
 'furniture': 24,
 'gardening tool': 18,
 'gemstone': 12,
 'geometric shape': 17,
 'green vegetable': 19,
 'hair colour': 18,
 'hat': 17,
 'healthcare profession': 23,
 'herb': 16,
 'human dwelling': 18,
 'infectious disease': 14,
 'injury': 20,
 'insect': 27,
 'jewellery': 14

In [5]:
all_words = set(category_norms['category.member'])
ts_df = ts_df.loc[ts_df.words.isin(all_words)]

In [6]:
norms = category_norms.groupby('category')['category.member'].apply(list).to_dict()
print(len(norms))

117


In [7]:
category_df = pd.DataFrame()
for category in norms:
    words = norms[category]
    c_df = ts_df.loc[ts_df.words.isin(words)]
    c_df['category'] = category
    category_df = pd.concat((category_df, c_df), ignore_index = True)
    
    

In [8]:
rel_df = category_df.loc[category_df.property == 'previous_link']
pol_df = category_df.loc[category_df.property == 'polarity']

top_categories = rel_df.sort_values(by = 'outputs_z',ascending = False).loc[rel_df.outputs_z > 1].category.unique()

rel_df = rel_df.loc[rel_df.category.isin(top_categories)]
pol_df = pol_df.loc[pol_df.category.isin(top_categories)]

In [9]:
top_categories = rel_df.sort_values(by = 'outputs_z',ascending = False).loc[rel_df.outputs_z > 1].category.unique()

In [10]:
joint_category_rel = rel_df.groupby(['year','category']).outputs_z.mean().reset_index().set_index(['year','category']).\
join(\
rel_df.groupby(['year','category']).outputs_z.std().reset_index().set_index(['year','category']),\
lsuffix='_mean',rsuffix = '_std').reset_index()

In [11]:
rel_df = category_df.loc[category_df.property == 'previous_link']
pol_df = category_df.loc[category_df.property == 'polarity']

In [12]:
len(rel_df.category.unique()), len(pol_df.category.unique())

(117, 117)

In [13]:
def get_growth_rate_word(category):
    c_df = rel_df.loc[rel_df.category == category]
    words = c_df.words.unique()
    if len(c_df) == 0:
        return []
    list_rows = []
    for word in words:
        w_df = c_df.loc[c_df.words == word]
        aggregate_moral_relevance = w_df.outputs_z.mean()
        is_moral = aggregate_moral_relevance > 0
                
        min_year = w_df.year.min()
        w_df['year'] = (w_df['year'] - min_year + 1) / 10
        model = smf.ols('outputs_z ~ year', data = w_df).fit() 
        list_rows.append( {'category':category,
                           'word':word,
               'r2': model.rsquared,
               'n':len(w_df),
               'fpvalue':model.f_pvalue,
                'intercept_coef': model.params.Intercept,
               'year_pvalue':model.pvalues.year,
               'year_coef':model.params.year,
               'is_moral':is_moral,
               'moral_relevance':aggregate_moral_relevance})
    result = pd.DataFrame(list_rows)
    return result
    
        
    
def get_growth_rate(category):
    c_df = rel_df.loc[rel_df.category == category]
    last_year_moral_relevance = c_df.loc[c_df.year == 2000].outputs_z.mean()
    if len(c_df) == 0:
        return []
    c_df = c_df.groupby('year')['outputs_z'].mean().reset_index()
    is_moral = True
    aggregate_moral_relevance = c_df.outputs_z.mean()
    if aggregate_moral_relevance < 0:
        is_moral = False
    min_year = c_df.year.min()
    c_df['year'] = (c_df['year'] - min_year + 1) / 10
    model = smf.ols('outputs_z ~ year', data = c_df).fit() 
    return {'category':category,
           'r2': model.rsquared,
           'n':len(c_df),
           'fpvalue':model.f_pvalue,
            'intercept_coef': model.params.Intercept,
           'year_pvalue':model.pvalues.year,
           'year_coef':model.params.year,
           'is_moral':is_moral,
           'moral_relevance':aggregate_moral_relevance,
           'final_moral_relevance':last_year_moral_relevance }
    
def plot_reg(category):
    c_df = rel_df.loc[rel_df.category == category]
    c_df = c_df.groupby('year')['outputs_z'].mean().reset_index()
    min_year = c_df.year.min()
    c_df['log_year'] = (c_df['year'] - min_year + 1)
    model = smf.ols('outputs_z ~ log_year', data = c_df).fit() 
    r2, coef = model.rsquared,model.params.log_year
    fig, axes = plt.subplots(1, 2, figsize = (10, 3))
    ax = axes[0]
    sns.regplot(data = c_df, x = 'log_year', y = 'outputs_z',ax =ax)
    ax.set_xlabel('time (relative)')
    ax.set_ylabel('moral relevance')
    ax.set_title(category + f'   R2: {round(r2,3)}, Coef: {round(coef,3)}')
    ax = axes[1]

    sns.lineplot(data = c_df, x = 'year', y = 'outputs_z',ax =ax)
    ax.set_xlabel('year')
    ax.set_ylabel('moral relevance')
    ax.set_title(category)
    sns.despine()
    plt.show()
        
    
    
    

In [14]:

reg = []
for c in norms:
    result = get_growth_rate(c)
    if len(result) == 0:
        continue
    reg.append(result)
reg = pd.DataFrame(reg)
len(reg)

117

In [15]:

coef = reg.groupby('category').year_coef.mean().to_dict()
r2 = reg.groupby('category').r2.mean().to_dict()

In [16]:
swow_df = pd.read_csv('./data/SWOWEN/moralized_v1_all.csv')
swow_df.columns

Index(['cue', 'total', 'cue_mfd2', 'response_mfd1', 'cue_mfd1',
       'response_mfd2', 'cue_hte', 'response_hte', 'response_emotion',
       'cue_emotion', 'cue_moral', 'response_moral', 'link', 'pos',
       'previous_link', 'lower', 'is_synonym', 'degree'],
      dtype='object')

In [17]:
swow_df['cue'] = swow_df['cue'].apply(lambda c: c.strip().lower())
swow_relevance = swow_df.groupby('cue').previous_link.mean().to_dict()
gpt4_df = pd.read_csv('data/SWOWEN/gpt-4o_prediction_df_cat.csv')
gpt4_df['word'] = gpt4_df['word'].apply(lambda c: c.strip().lower())
gpt4_relevance = gpt4_df.groupby('word').gpt_score.mean().to_dict()
len(gpt4_relevance)

1967

In [18]:
rel_df['swow_relevance'] = rel_df.words.apply(lambda w: None if w not in swow_relevance\
                                                       else np.log(swow_relevance[w] * 100 + 1))
rel_df['gpt_relevance'] = rel_df.words.apply(lambda w: None if w not in gpt4_relevance\
                                                       else np.log(gpt4_relevance[w] * 100 + 1))

In [19]:
category_swow_relevance = rel_df[['category','words','swow_relevance']].dropna().\
groupby('category')[['swow_relevance']].mean().to_dict()
category_gpt_relevance = rel_df[['category','words','gpt_relevance']].dropna().\
groupby('category')[['gpt_relevance']].mean().to_dict()


In [20]:
reg['swow_relevance'] = reg['category'].apply(lambda c : category_swow_relevance['swow_relevance'][c])
reg['gpt_relevance'] = reg['category'].apply(lambda c : category_gpt_relevance['gpt_relevance'][c]
                                             if c in category_gpt_relevance['gpt_relevance'] else None)
reg.dropna()

category        r2   n       fpvalue  intercept_coef  \
0     academic subject  0.747526  16  1.552550e-05        0.188326   
1      alcoholic drink  0.809026  16  2.126897e-06       -0.661386   
2               animal  0.105152  16  2.204541e-01       -0.678038   
3             art form  0.886088  16  5.494554e-08       -0.948935   
4    artistic movement  0.012758  16  6.770453e-01       -1.021080   
..                 ...       ...  ..           ...             ...   
112        water sport  0.299059  13  5.311128e-02       -1.123662   
113             weapon  0.699340  16  5.423248e-05        0.082738   
114            weather  0.574707  16  6.665873e-04       -0.930032   
115    wind instrument  0.828397  16  9.958840e-07       -0.355988   
116       winter sport  0.505060   9  3.187637e-02       -0.506436   

      year_pvalue  year_coef  is_moral  moral_relevance  \
0    1.552550e-05  -0.014912      True         0.074995   
1    2.126897e-06  -0.011881     False        -0.751680   
2    2.204541e-01   0.001747     False        -0.664762   
3    5.494554e-08  -0.011417     False        -1.035701   
4    6.770453e-01  -0.001207     False        -1.030250   
..            ...        ...       ...              ...   
112  5.311128e-02  -0.009372     False        -1.182996   
113  5.423248e-05   0.016883      True         0.211047   
114  6.665873e-04  -0.004089     False        -0.961107   
115  9.958840e-07  -0.038857     False        -0.651301   
116  3.187637e-02  -0.053888     False        -0.727378   

     final_moral_relevance  swow_relevance  gpt_relevance  
0                 0.042862        0.879112       2.786485  
1                -0.802043        0.224454       2.734541  
2                -0.639410        0.465288       2.484928  
3                -1.100291        0.179636       2.318282  
4                -0.901535        0.000000       2.555960  
..                     ...             ...            ...  
112              -1.179340        0.000000       1.791759  
113               0.246359        1.718627       2.943450  
114              -0.969227        0.000000       1.918557  
115              -0.945741        0.000000       2.360012  
116              -0.799858        0.000000       2.397895  

[117 rows x 12 columns]

In [21]:
from adjustText import adjust_text


In [22]:
spearmanr(reg.moral_relevance, reg.swow_relevance) #GNN versus SWOW

SignificanceResult(statistic=0.7443269554769868, pvalue=6.783160102687718e-22)

In [23]:
spearmanr(reg.moral_relevance, reg.gpt_relevance) #GNN versus gpt4o

SignificanceResult(statistic=0.5736679230175207, pvalue=1.3731930374439927e-11)

In [24]:
spearmanr(reg.swow_relevance, reg.gpt_relevance) #SWOW versus gpt4o

SignificanceResult(statistic=0.5652979745118057, pvalue=3.134299618014014e-11)

In [25]:
import pingouin as pg

x = reg.moral_relevance
y = reg.swow_relevance
corr1 = pg.corr(x, y,method = 'spearman').round(3)
x = reg.gpt_relevance
corr2 = pg.corr(x, y,method = 'spearman').round(3)

errors1 = [corr1['CI95%'].values[0][0],corr1['CI95%'].values[0][1]]

errors2 = [corr2['CI95%'].values[0][0],corr2['CI95%'].values[0][1]]
bars1 = corr1['r'].values[0]
bars2 = corr2['r'].values[0]

errors1 = np.abs(np.array(errors1) - bars1)
errors2 = np.abs(np.array(errors2) - bars2)


In [26]:
corr1,corr2

(            n      r         CI95%  p-val  power
 spearman  117  0.744  [0.65, 0.82]    0.0    1.0,
             n      r         CI95%  p-val  power
 spearman  117  0.565  [0.43, 0.68]    0.0    1.0)

In [27]:

fig,ax = plt.subplots(figsize = (20, 10))
g = sns.scatterplot(data = reg, x = 'moral_relevance', y = 'year_coef',alpha = 0.7,
                   s = 100)
g.legend_ = None
groups = ['disease','science','supernatural being']
annotations = []
for i, row in reg.iterrows():
    if row['category'] not in top_categories:
        continue
    annotations.append(g.annotate(row['category'], (row['moral_relevance'], row['year_coef']),
                color = 'black',size = 19))
    
    
adjust_text(annotations,expand=(1, 1.2),arrowprops=dict(arrowstyle='->', color='blue'))
g.set_xlabel('Estimated degree of moral relevance',fontsize = 25)
g.set_ylabel('Estimated rate of moralization',fontsize = 25)

plt.axhline(y = 0,linestyle = '--')
plt.axvline(x= 0,linestyle = '--')
ax.set_xticklabels(ax.get_xticklabels(), fontsize = 25)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 25)
ax.text(x = -1.4, y = -0.055, s = 'Low', color = 'Black',fontsize = 25,
       bbox=dict(facecolor='lightgrey', edgecolor='lightgrey', boxstyle='round,pad=0.1'))

ax.text(x = -1.4, y = 0.055, s = 'High', color = 'Green',fontsize = 25,
       bbox=dict(facecolor='lightgreen', edgecolor='lightgreen', boxstyle='round,pad=0.1'))


ax.text(x = 2.5, y = -0.055, s = 'High', color = 'Green',fontsize = 25,
       bbox=dict(facecolor='lightgreen', edgecolor='lightgreen', boxstyle='round,pad=0.1'))

left, bottom, width, height = [.7, 0.78, 0.2, 0.3]
ax2 = fig.add_axes([left, bottom, width, height])


ax2.bar([0,1], [bars1,bars2],capsize = 5,
                  tick_label = ['Reconstruction model','GPT-4o'],
                  color = ['tab:orange','tab:blue'],
                  yerr = [errors1,errors2],
              width = 0.5
                       )


ax2.set_ylabel('Correlation with empirical\nmoral relevance',fontsize = 14)
ax2.set_xticklabels(ax2.get_xticklabels(), fontsize = 14)
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize = 14)

sns.despine()
plt.savefig('notebooks/figs/figure4_coha.pdf',dpi = 500, bbox_inches = 'tight')
